<ul class="breadcrumb">
  <li><a href="2.1.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="2.3.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font><br> 
  <font size="5" face="Georgia">(with Maternal Effects)</font> 

</div>

In [2]:
using DataFrames
using JWAS: MT,Datasets

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multivariate Linear Additive Genetic Model</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [2]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [3]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [4]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [5]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [7]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [6]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [12]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [13]:
model2    = MT.build_model(model_equations,R);

In [14]:
MT.set_covariate(model1,"age")

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [15]:
ped = MT.get_pedigree(pedfile);

In [16]:
MT.set_random(model2,"Animal", ped,G)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [17]:
out2 = MT.runMCMC(model2,data,chain_length=50000,printout_frequency=10000);

at sample: 10000
Residual covariance matrix: 
[10.076070697796817 2.014053662507801
 2.014053662507801 0.9907440781291249]

at sample: 20000
Residual covariance matrix: 
[9.795159647202667 1.967417772112444
 1.967417772112444 0.9954529847492887]

at sample: 30000
Residual covariance matrix: 
[10.029809802614125 1.9265927901393582
 1.9265927901393582 0.9784010204519306]

at sample: 40000
Residual covariance matrix: 
[10.071169109358841 1.9570225392422889
 1.9570225392422889 0.9909602442557353]

at sample: 50000
Residual covariance matrix: 
[10.011037605962265 1.9546467419619271
 1.9546467419619271 0.989539181215114]



<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. The Multivariate Linear Additive Genetic Model with Maternal Effects</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [34]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [35]:
d2 = [data[3:4,:]  DataFrame(Dam = ["D1","D1"])]

,Animal,BW,CW,age,sex,Dam
1,O1,150.0,13.0,3,M,D1
2,O3,40.0,5.0,4,F,D1


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [36]:
model_equations = "BW = intercept + age + sex + Animal+ Dam;
                   CW = intercept + age + sex + Animal";

In [37]:
model5 = MT.build_model(model_equations,R);

In [38]:
MT.set_covariate(model1,"age")

In [39]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
MT.set_random(model5,"Animal Dam", ped,G0)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [ ]:
out5=MT.runMCMC(model5,d2,chain_length=50000,printout_frequency=10000);